In [ ]:
import requests, shutil, time, cv2
from bs4 import BeautifulSoup
import numpy as np
from keras.models import load_model
from preprocessBatch import preprocessing
from utilities import one_hot_decoding

allowedChars = 'ACDEFGHJKLNPQRTUVXYZ2346789';
CAPTCHA_IMG = "captcha.jpg"
PROCESSED_IMG = "preprocessing.jpg"
FOLDER = "captcha/"
CSV_FILE = "auto-train.csv"

In [ ]:
print('model loading...')
model = load_model("twse_cnn_model.hdf5")
print('loading completed')

In [ ]:
i = 5039

while i < 5500:
    resp = requests.get("https://bsr.twse.com.tw/bshtm/bsMenu.aspx")
    soup = BeautifulSoup(resp.text, 'html.parser')
    img_tags = soup.select("#Panel_bshtm img")
    src = img_tags[0].get('src')

    resp = requests.get("https://bsr.twse.com.tw/bshtm/" + src, stream=True)
    if resp.status_code == 200:
        with open(CAPTCHA_IMG, 'wb') as f:
            resp.raw.decode_content = True
            shutil.copyfileobj(resp.raw, f)
            
    preprocessing(CAPTCHA_IMG, PROCESSED_IMG)
    train_data = np.stack([np.array(cv2.imread(PROCESSED_IMG))/255.0])
    prediction = model.predict(train_data)
    
    predict_captcha = one_hot_decoding(prediction, allowedChars)

    payload = {}
    acceptable_input = ['__VIEWSTATE', '__VIEWSTATEGENERATOR', '__EVENTVALIDATION', 'RadioButton_Normal',
                        'TextBox_Stkno', 'CaptchaControl1', 'btnOK']

    inputs = soup.select("input")
    for elem in inputs:
        if elem.get("name") in acceptable_input:
            if elem.get("value") != None:
                payload[elem.get("name")] = elem.get("value")
            else:
                payload[elem.get("name")] = ""
                
    payload['TextBox_Stkno'] = '2330'
    payload['CaptchaControl1'] = predict_captcha
    
    resp = requests.post("https://bsr.twse.com.tw/bshtm/bsMenu.aspx", data=payload)
    if '驗證碼錯誤!' in resp.text:
        print('驗證碼錯誤, predict_captcha: ' + predict_captcha)
    elif '驗證碼已逾期!' in resp.text:
        print('驗證碼已逾期, predict_captcha: ' + predict_captcha)
    elif 'HyperLink_DownloadCSV' in resp.text:
        # add to label\n",
        i += 1
        print("success, add " + str(i) + ".jpg with captcha: " + predict_captcha)
        shutil.copyfile(CAPTCHA_IMG, FOLDER + str(i) + ".jpg")
        with open(CSV_FILE,'a') as fd:
            fd.write(predict_captcha + "\n")
            
    time.sleep(1)

print("completed")